In [ ]:
# Requirements for this notebook to run: pandas, geopandas, shapely, pyproj
import concurrent.futures
import os
from datetime import datetime
import time
import json

import pandas
from pandas import Series
import geopandas
from shapely.geometry import shape
import contextily as ctx

from eocalc.context import Pollutant
from eocalc.methods.base import DateRange
from eocalc.methods.temis import TropomiMonthlyMeanAggregator

In [ ]:
directory = "data/regions"
regions = {}

for filename in os.listdir(directory):
    if filename.endswith(".geo.json") or filename.endswith(".geojson"):
        with open(f"{directory}/{filename}", 'r') as geojson_file:
            regions[filename] = shape(json.load(geojson_file)["geometry"])

del regions["europe.geo.json"] # This takes too long...

# Remove regions not covered
remove = [filename for filename, region in regions.items() if not TropomiMonthlyMeanAggregator().covers(region)]
for filename in remove: del regions[filename]

regions

In [ ]:
results = {} # results will be put here as results[<filename>][<data>]
start = datetime.now()

for filename, region in regions.items():
    results[filename] = TropomiMonthlyMeanAggregator().run(region, DateRange(start='2018-08-01', end='2018-08-31'), Pollutant.NO2)
    print(f"Done with region represented by file '{filename}'")

print(f"All finished in {datetime.now()-start}.")

In [ ]:
for filename in regions:
    table = results[filename][TropomiMonthlyMeanAggregator.TOTAL_EMISSIONS_KEY]
    print(f"Total emissions in region {filename}: {table.iloc[-1, 0]:.2f}kt {Pollutant.NO2.name} (±{table.iloc[-1, 1]:.1f}%)")

In [ ]:
for filename in regions:
    gridded_result = results[filename][TropomiMonthlyMeanAggregator.GRIDDED_EMISSIONS_KEY]
    gridded_result.plot(f"Total {Pollutant.NO2.name} emissions [kg]", figsize=(20, 20), legend=True, legend_kwds={'label': f"Emissions in {filename} [kg]", 'orientation': "horizontal"})

In [ ]:
for filename in regions:
    gridded_result = results[filename][TropomiMonthlyMeanAggregator.GRIDDED_EMISSIONS_KEY]
    ctx.add_basemap(gridded_result.to_crs(epsg=3857).plot(f"Total {Pollutant.NO2.name} emissions [kg]", figsize=(20, 20), alpha=0.7, edgecolor='k'))